In [234]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.pydantic_v1 import Field
from langchain_openai import ChatOpenAI


In [12]:
from google.colab import userdata
openai_api_key = userdata.get("OPENAI_API_KEY")
together_api_key = userdata.get("TOGETHER_API_KEY")

In [283]:
class Persons(BaseModel):
    names: list[str] = Field(description="person names")


In [14]:
example_output = """Example 1: 
Input: text = "Встреча с писательницей Сюзанной Кулешовой Презентация книги «Последний глоток божоле на двоих». Кулешова Сюзанна Марковна, член Союза писателей Санкт-Петербурга. Закончила Горный институт, работала палеонтологом, педагогом, редактором. Первая публикация в газете «Ленинские искры». [буквоед]"
Output: ["Сюзанной Кулешовой", "Кулешова Сюзанна Марковна"]
Exmaple 2
Input text: "Фестиваль Поэзия и вино. 20:00 - о Любви. Милена Райт и Костя Гафнер 21:10 - Открытый Микрофон [соб. инф.]"
Output: ["Милена Райт", "Костя Гафнер"]
Example 3
Input text: "В рамках выставки «Максим Винавер. Пора возвращаться домой…». Спектакль по пьесе Максима Винавера «11 сентября». [музей]"
Output: ["Максима Винавера"]
Example 4
Input text: "Встреча с писателем Александром Чудаковым. Александр Чудаков - писатель, автор книги «Последний герой». [соб. инф.]"
Output: ["Александром Чудаковым", "Александр Чудаков"]
"""

prompt = """You are a Named Entity Recognition (NER) system designed to process Russian text. Your task is to extract the names of real people mentioned in the text. You are given the text. Follow these steps:
                1. Read the text and identify the names of the people mentioned in the text. 
                2. Consider only the names of people if they describe a person. We don't need titles of works of art or names of organizations if they contain the names of people.
                3. Return the list of these names in the same form as they appear in the text. Don't change the form of any name!
                4. Be sure that all occurences of the persons names in the text are included in the list even if they are mentioned several times. 
                Return only the list of names without any additional comments or formatting. 
            """

In [30]:
import pandas as pd
import json
data = []
with open("data/labeled_data_1000.json1", "r") as f:
  d = f.readlines()
  for each in d:
    data.append(json.loads(each))
df = pd.DataFrame.from_records(data)

In [31]:
df

,id,text,meta,annotation_approver,labels
0,2201,Вечер поэта Томаса Венцлова (США). Презентация...,{},None,"[[12, 27, PERSON]]"
1,2202,Лекция Александра Степанова «Genius loci. Смыс...,{},None,"[[7, 27, PERSON]]"
2,3134,В магазине книг «Фаренгейт 451» писатель Роман...,{},None,"[[41, 53, PERSON]]"
3,3135,Настройки / Settings. Александр Ильянен (лекци...,{},None,"[[50, 68, PERSON], [22, 39, PERSON]]"
4,2203,21 марта во Всемирный День Поэзии состоится Те...,{},None,[]
...,...,...,...,...,...
995,3195,Творческая встреча с русским и американским ху...,{},None,"[[75, 92, PERSON], [193, 207, PERSON], [712, 7..."
996,3196,"Выставка-исследование ""Постсоветское Красное З...",{},None,"[[1439, 1451, PERSON], [1512, 1524, PERSON], [..."
997,3197,"Презентация книги Виктора Тихомирова ""ПРО ШПИО...",{},None,"[[18, 36, PERSON], [126, 142, PERSON]]"
998,3198,"На заседании клуба ""Берггассе, 19"" состоится т...",{},None,"[[68, 79, PERSON]]"


In [14]:
import ast

def convert_to_list_of_tuples(data_str):
    try:
        return ast.literal_eval(data_str)
    except ValueError as e:
        print(f"Error converting string to list: {e}")
        return []  # Return an empty list in case of error

df['labels'] = df['labels'].apply(lambda x: x[0] if x else x)

In [22]:
text = 'Литературнвый вечер "Мы все учились понемногу...". Участвуют: Вероника Капустина, Александр Гуревич, Наталья Перевезенцева, Татьяна Алфёрова, Нина Савушкина, Борис Григорин, Вадим Пугач, Александр Фролов. В программе: роман в стихах "Богатая Лиза", случаи и анекдоты, как бы переводы с латиноамериканского и другого, собственно стихи.'

In [1]:
import pandas as pd
df = pd.read_csv("data/dataset_with_models_1000_31_05_2024.csv")


,Unnamed: 0,id,text,labels,mult_model,rus_ner_model,roberta_large,spacy,gpt,gpt4
0,0,2201,Вечер поэта Томаса Венцлова (США). Презентация...,"[[12, 27, 'PERSON']]","[(12, 28, 'PERSON'), (29, 32, 'GPE'), (53, 71,...","[(12, 28, 'PER')]","[[12, 27, 'PER'], [71, 72, 'LOC'], [75, 78, 'O...","[[12, 27, 'PER'], [29, 32, 'LOC']]","[(12, 27, 'PERSON')]","[(12, 27, 'PERSON')]"
1,1,2202,Лекция Александра Степанова «Genius loci. Смыс...,"[[7, 27, 'PERSON']]","[(7, 28, 'PERSON'), (28, 54, 'WORK_OF_ART')]","[(7, 28, 'PER')]","[[7, 27, 'PER']]","[[7, 27, 'PER']]","[(7, 27, 'PERSON')]","[(7, 27, 'PERSON')]"
2,2,3134,В магазине книг «Фаренгейт 451» писатель Роман...,"[[41, 53, 'PERSON']]","[(17, 30, 'WORK_OF_ART'), (41, 54, 'PERSON'), ...","[(41, 54, 'PER'), (197, 214, 'ORG')]","[[41, 53, 'PER'], [55, 61, 'LOC'], [197, 212, ...","[[41, 53, 'PER'], [55, 61, 'LOC'], [79, 87, 'P...","[(41, 53, 'PERSON')]","[(41, 53, 'PERSON')]"
3,3,3135,Настройки / Settings. Александр Ильянен (лекци...,"[[50, 68, 'PERSON'], [22, 39, 'PERSON']]","[(22, 40, 'PERSON'), (50, 69, 'PERSON')]","[(22, 40, 'PER'), (50, 69, 'PER')]","[[22, 39, 'PER'], [50, 68, 'PER']]","[[22, 39, 'PER'], [50, 68, 'PER']]","[(22, 39, 'PERSON'), (50, 68, 'PERSON')]","[(22, 39, 'PERSON'), (50, 68, 'PERSON')]"
4,4,2203,21 марта во Всемирный День Поэзии состоится Те...,[],"[(0, 9, 'DATE'), (12, 34, 'EVENT'), (44, 74, '...",[],"[[0, 8, 'ORG'], [12, 33, 'ORG'], [44, 584, 'OR...","[[0, 8, 'DATE'], [44, 66, 'LOC'], [67, 72, 'LO...","[(44, 72, 'PERSON')]",[]
...,...,...,...,...,...,...,...,...,...,...
995,995,3195,Творческая встреча с русским и американским ху...,"[[75, 92, 'PERSON'], [193, 207, 'PERSON'], [71...","[(21, 29, 'NORP'), (31, 44, 'NORP'), (75, 94, ...","[(75, 94, 'PER'), (193, 208, 'PER'), (391, 406...","[[75, 92, 'PER'], [107, 113, 'LOC'], [134, 143...","[[75, 92, 'PER'], [107, 113, 'LOC'], [134, 143...",NaN,NaN
996,996,3196,"Выставка-исследование ""Постсоветское Красное З...","[[1439, 1451, 'PERSON'], [1512, 1524, 'PERSON'...",[],[],"[[95, 116, 'LOC'], [1081, 1088, 'ORG'], [1094,...","[[23, 50, 'ORG'], [62, 75, 'ORG'], [95, 116, '...",NaN,NaN
997,997,3197,"Презентация книги Виктора Тихомирова ""ПРО ШПИО...","[[18, 36, 'PERSON'], [126, 142, 'PERSON']]","[(18, 37, 'PERSON'), (37, 50, 'WORK_OF_ART'), ...","[(18, 37, 'PER'), (126, 143, 'PER')]","[[18, 36, 'PER'], [126, 142, 'PER']]","[[18, 36, 'PER'], [126, 142, 'PER']]",NaN,NaN
998,998,3198,"На заседании клуба ""Берггассе, 19"" состоится т...","[[68, 79, 'PERSON']]","[(19, 35, 'ORG'), (68, 79, 'PERSON')]","[(20, 33, 'ORG'), (68, 79, 'PER')]","[[68, 79, 'PER']]","[[20, 29, 'ORG'], [68, 79, 'PER']]",NaN,NaN


In [ ]:
import ast

def convert_to_list_of_tuples(data_str):
    try:
        return ast.literal_eval(data_str)
    except ValueError as e:
        return []  # Return an empty list in case of error

df['labels'] = df['labels'].apply(convert_to_list_of_tuples)
df['mult_model'] = df['mult_model'].apply(convert_to_list_of_tuples)
df['rus_ner_model'] = df['rus_ner_model'].apply(convert_to_list_of_tuples)
df['spacy'] = df['spacy'].apply(convert_to_list_of_tuples)
df['gpt'] = df['gpt'].apply(convert_to_list_of_tuples)
df['gpt4'] = df['gpt4'].apply(convert_to_list_of_tuples)
df["roberta_large"] = df["roberta_large"].apply(convert_to_list_of_tuples)

In [15]:
import json
import openai
from pydantic import BaseModel, Field

# Create client
client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=together_api_key,
)

In [134]:
def extract_persons(text):
    chat_completion = client.chat.completions.create(
        #model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        model="togethercomputer/CodeLlama-34b-Instruct",
        response_format={"type": "json_object", "schema": Persons.model_json_schema()},
        messages=[
              {
                "role": "system",
                "content": f"""{prompt}.\n
                Example: {example_output}
                """,
            },
            {
                "role": "user",
                "content": f"Extract person names from this text: {text}",
            },
        ],
    )
    result = json.loads(chat_completion.choices[0].message.content)
    if "names" in result:
        return result["names"]
    else:
        return []

In [17]:
def find_name_positions(text, names):
    results = []
    start_index = 0

    for name in names:
        start = text.find(name, start_index)
        if start != -1:
            end = start + len(name)
            results.append((start, end, 'PERSON'))
            start_index = end  # Prevent re-finding the same name

    return results

# Example usage
text = "Очередная встреча проекта «Открытая читка – юность» – литературный салон, где чтение вслух есть способ общения и творческого самовыражения отрока, тинэйджера, школьника – «Человек Читающий». Творческое объединение «Digital Chehov» в рамках проектов «Человек Читающий» и «Читающие дети» при поддержке «Открытой гостиной», издательств: «Питер», «Самокат», «Нигма», «Белая ворона» и портала «Литературно» открывает новый сезон «Открытой Читки – юность»! В этот день соберутся любители литературного чтения от 7 до 17 лет. Ребята будут читать вслух собственные сочинения и чужие произведения: проза или поэзия, «по листочку» или наизусть – выбор способа выражения остаётся за участниками. Главное – свободное объяснение со своими литературными предпочтениями. На «Читке» будем снимать видеоролики и фото для проекта «Человек Читающий». Завершится творческий вечер выбором трёх лучших чтецов: ребята получат подарки от издательств: «Питер», «Самокат», «Нигма», «Белая ворона». Все юные смельчаки гарантированно получат внимание и аплодисменты слушателей. «Открытая читка» – это не просто возможность особенным образом поделиться своими мыслями с окружающими, но и шанс найти неравнодушных к твоим интересам друзей. Читка – это способ литературной коммуникации – выход в оффлайн, возвращение к «бородатой» традиции чтения вслух перед аудиторией, которая сейчас опять становится востребованной. Куратор − Черток Анна [маяковка]"
names = ['Черток Анна']

positions = find_name_positions(text, names)
print(positions)


[(1398, 1409, 'PERSON')]


In [282]:
df.to_csv("data/dataset_with_models_1000_31_05_2024.csv")

In [281]:
df[990:1000]

,id,text,labels,mult_model,rus_ner_model,roberta_large,spacy,gpt4,mistral,gpt4o,gpt3.5-31-05-2024,gpt4o_json
990,3190,1000 солнц. Выступление жёстких брутальных и э...,[],"[(0, 5, CARDINAL)]",[],"[[0, 10, ORG]]",[],[],"[(0, 10, PERSON)]",[],"[(0, 10, PERSON)]",[]
991,3191,"Презентация нового, 69-го номера Митиного Журн...","[[100, 117, PERSON], [166, 180, PERSON], [458,...","[(20, 22, ORDINAL), (23, 26, ORDINAL), (33, 51...","[(33, 51, ORG), (100, 119, PER), (166, 180, PE...","[[33, 48, ORG], [100, 117, PER], [166, 180, PE...","[[33, 49, PER], [100, 117, PER], [166, 180, PE...","[(100, 117, PERSON)]","[(100, 117, PERSON), (458, 469, PERSON), (498,...","[(100, 117, PERSON), (600, 619, PERSON), (621,...","[(600, 619, PERSON), (621, 639, PERSON), (642,...","[(100, 117, PERSON), (166, 180, PERSON), (458,..."
992,3192,Литературный вечер. У нас в гостях поэт Тимофе...,"[[40, 59, PERSON]]","[(40, 59, PERSON)]","[(40, 59, PER)]","[[40, 59, PER]]","[[40, 59, PER]]","[(40, 59, PERSON)]","[(40, 59, PERSON)]","[(40, 59, PERSON)]","[(40, 59, PERSON)]","[(40, 59, PERSON)]"
993,3193,«И будет сладкая тревога расти при имени твоем...,"[[49, 64, PERSON], [385, 401, PERSON], [432, 4...","[(49, 65, PERSON), (249, 263, PERSON), (385, 4...","[(49, 65, PER), (249, 263, PER), (385, 403, PE...","[[49, 64, PER], [249, 263, PER], [385, 401, PE...","[[49, 64, PER], [249, 256, LOC], [257, 263, LO...","[(49, 64, PERSON), (249, 263, PERSON), (416, 4...","[(49, 64, PERSON), (249, 263, PERSON), (484, 5...","[(49, 64, PERSON), (249, 263, PERSON), (385, 4...","[(49, 64, PERSON), (1299, 1315, PERSON)]","[(49, 64, PERSON), (249, 263, PERSON), (385, 4..."
994,3194,К Дню памяти Иосифа Бродского. Встреча с писат...,"[[89, 107, PERSON], [134, 150, PERSON], [254, ...","[(13, 31, PERSON), (89, 108, PERSON), (128, 16...","[(13, 31, PER), (89, 108, PER), (254, 271, PER...","[[13, 29, PER], [89, 107, PER], [254, 270, PER...","[[13, 29, PER], [89, 107, PER], [134, 150, PER...","[(13, 29, PERSON), (89, 107, PERSON), (254, 27...",[],"[(13, 29, PERSON), (89, 107, PERSON), (134, 15...","[(13, 29, PERSON), (89, 107, PERSON), (254, 27...","[(13, 29, PERSON), (89, 107, PERSON), (134, 15..."
995,3195,Творческая встреча с русским и американским ху...,"[[75, 92, PERSON], [193, 207, PERSON], [712, 7...","[(21, 29, NORP), (31, 44, NORP), (75, 94, PERS...","[(75, 94, PER), (193, 208, PER), (391, 406, PE...","[[75, 92, PER], [107, 113, LOC], [134, 143, LO...","[[75, 92, PER], [107, 113, LOC], [134, 143, LO...","[(75, 92, PERSON), (193, 207, PERSON), (391, 4...","[(193, 207, PERSON)]","[(75, 92, PERSON), (193, 207, PERSON), (391, 4...","[(75, 92, PERSON), (193, 207, PERSON), (862, 8...","[(75, 92, PERSON), (193, 207, PERSON), (391, 4..."
996,3196,"Выставка-исследование ""Постсоветское Красное З...","[[1439, 1451, PERSON], [1512, 1524, PERSON], [...",[],[],"[[95, 116, LOC], [1081, 1088, ORG], [1094, 112...","[[23, 50, ORG], [62, 75, ORG], [95, 116, LOC],...","[(1439, 1451, PERSON), (1512, 1524, PERSON), (...","[(1439, 1451, PERSON), (1512, 1524, PERSON), (...","[(1439, 1451, PERSON), (1512, 1524, PERSON), (...","[(1439, 1451, PERSON), (1512, 1524, PERSON), (...","[(1439, 1451, PERSON), (1512, 1524, PERSON), (..."
997,3197,"Презентация книги Виктора Тихомирова ""ПРО ШПИО...","[[18, 36, PERSON], [126, 142, PERSON]]","[(18, 37, PERSON), (37, 50, WORK_OF_ART), (75,...","[(18, 37, PER), (126, 143, PER)]","[[18, 36, PER], [126, 142, PER]]","[[18, 36, PER], [126, 142, PER]]","[(18, 36, PERSON), (126, 142, PERSON)]","[(18, 36, PERSON)]","[(18, 36, PERSON), (126, 142, PERSON)]","[(18, 36, PERSON), (126, 142, PERSON)]","[(18, 36, PERSON), (126, 142, PERSON)]"
998,3198,"На заседании клуба ""Берггассе, 19"" состоится т...","[[68, 79, PERSON]]","[(19, 35, ORG), (68, 79, PERSON)]","[(20, 33, ORG), (68, 79, PER)]","[[68, 79, PER]]","[[20, 29, ORG], [68, 79, PER]]","[(68, 79, PERSON)]","[(68, 79, PERSON)]","[(68, 79, PERSON)]","[(68, 79, PERSON)]","[(68, 79, PERSON)]"
999,3199,В литературном мини-отеле «

In [232]:
example = """
Example 1: 
Input: text = "Встреча с писательницей Сюзанной Кулешовой Презентация книги «Последний глоток божоле на двоих». Кулешова Сюзанна Марковна, член Союза писателей Санкт-Петербурга. Закончила Горный институт, работала палеонтологом, педагогом, редактором. Первая публикация в газете «Ленинские искры». [буквоед]"
Output: ["Сюзанной Кулешовой", "Кулешова Сюзанна Марковна"]
Exmaple 2
Input text: "Фестиваль Поэзия и вино. 20:00 - о Любви. Милена Райт и Костя Гафнер 21:10 - Открытый Микрофон [соб. инф.]"
Output: ["Милена Райт", "Костя Гафнер"]
Example 3
Input text: "В рамках выставки «Максим Винавер. Пора возвращаться домой…». Спектакль по пьесе Максима Винавера «11 сентября». [музей]"
Output: ["Максима Винавера"]
"""


In [235]:
parser = PydanticOutputParser(pydantic_object=Persons)

structured_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a Named Entity Recognition (NER) system designed to process Russian text. Your task is to extract the names of real people mentioned in the text. You are given the text. Follow these steps:\n
            1. Read the text and identify the names of the people mentioned in the text. \n
            2. Consider only the names of people if they describe a person. We don't need titles of works of art or names of organizations if they contain the names of people.\n
            3. Return the list of these names in the same form as they appear in the text. Don't change the form of any name!\n
            4. Be sure that all occurences of the names in the text are included in the list even if they are mentioned several times. \n
            Return only the list of names without any additional comments or formatting. 
            """,
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                template="Input text: {text}."
                "\n{format_instructions}\nExample output: {example}",
                input_variables=["text", "example_output"],
                partial_variables={
                    "format_instructions": parser.get_format_instructions(),
                },
            )
        ),
    ]
)

In [242]:
llm = ChatOpenAI(
    max_tokens=4096,
    temperature=0,              
    model_name="gpt-4o-2024-05-13",
    tags=["toc_classifier"],
    timeout=300,
    verbose=True,
    openai_api_key=openai_api_key,
)

chain = structured_prompt | llm | parser

### Query with parser example

In [269]:
for i, row in df.iterrows():
    text = row["text"]
    result = chain.invoke({"text": text, "example": example})
    if result.names:
        predicted_labels = find_name_positions(text, result.names)
        df.at[i, "gpt4o_json"] = predicted_labels
    else:
        df.at[i, "gpt4o_json"] = []


Text: Евгений Кучинов. Презентация книги А. Святогора «Поэтика. Биокосмизм. (А)теология» (+ лекция: Анархо-космизм Братьев Гординых и Александра Святогора). Встреча будет посвящена вулканам, различным видам зверья, румяной технике и краснощеким словам, органам, выпрыгивающим из тел, и аромату революции — то есть такому странному и малоизвестному феномену, который известен как анархо-космизм. Здесь переплетутся вопросы о сущности магии и техники, о безвластии как выходе в космос, о языках космического общения, об аффектах выхода за пределы человеческого вида, о Земле как космическом корабле и о тех отделах, которые необходимо посетить каждому, кто окажется в Стране Анархии. Нашими спутниками будут: Аба и Вольф Гордины (Братья Гордины, Бр. Гордин), Н. Гордый, Бэоби, Александр Агиенко, Святогор, Пересвет-Пересветов и другие инопланетяне. Ожидается, что разговор будет предельно серьезным. Книга Алекандра Святогора: http://common.place/svyatogor/ Говорящая голова: Евгений Кучинов — философ,

### Simple API query

In [183]:
for i, row in df[:5].iterrows():
    text = row["text"]
    print(f"Text: {text}")
    print(f"Labels: {row['labels']}")
    result = extract_persons(text)
    if result:
        predicted_labels = find_name_positions(text, result)
        print(f"Predicted labels: {predicted_labels}")
        df.at[i, "gpt4"] = predicted_labels
    print("")


Text: Вечер поэта Томаса Венцлова (США). Презентация книги "Гранёный воздух" М.: ОГИ, 2002.
Labels: [[12, 27, 'PERSON']]
Predicted labels: [(53, 54, 'PERSON'), (64, 65, 'PERSON'), (70, 71, 'PERSON')]

Text: Лекция Александра Степанова «Genius loci. Смысл образа» [нг, регистрация https://newholland.timepad.ru/event/727346/]
Labels: [[7, 27, 'PERSON']]
Predicted labels: [(56, 57, 'PERSON'), (62, 63, 'PERSON'), (65, 66, 'PERSON'), (68, 69, 'PERSON'), (72, 73, 'PERSON'), (116, 117, 'PERSON')]

Text: В магазине книг «Фаренгейт 451» писатель Роман Сенчин (Москва), автор романов «Елтышевы», «Лёд под ногами», «Минус» и многих других, расскажет о своей новой книге «Зона затопления» о строительстве Богучанской ГЭС. [соб. инф.]
Labels: [[41, 53, 'PERSON']]
Predicted labels: [(214, 215, 'PERSON'), (216, 217, 'PERSON'), (221, 222, 'PERSON'), (224, 225, 'PERSON')]

Text: Настройки / Settings. Александр Ильянен (лекция). Сергей Соколовский (чтение). [серия]
Labels: [[50, 68, 'PERSON'], [22, 39, 'PERS

In [52]:
with open("data/prompt2.txt", "r") as f:
  prompt = f.read()

In [140]:
prompt

'You are a Named Entity Recognition (NER) system designed to process Russian text. Your task is to extract the names of real people mentioned in the text. You are given the text. Follow these steps:\n\n1. Read the text and identify the names of the people mentioned in the text. \n2. Consider only the names of people if they describe a person. We don\'t need titles of works of art or names of organizations if they contain the names of people.\n3. Return the list of these names in the same form as they appear in the text. Don\'t change the form of any name!\n4. Be sure that all occurences of the names in the text are included in the list even if they are mentioned several times. \nReturn only the list of names without any additional comments or formatting. \n\nExample 1: \nInput: text = "Встреча с писательницей Сюзанной Кулешовой Презентация книги «Последний глоток божоле на двоих». Кулешова Сюзанна Марковна, член Союза писателей Санкт-Петербурга. Закончила Горный институт, работала пале

In [187]:
def extract_persons(text):
    openai.api_key = openai_api_key

    response = openai.chat.completions.create(
        #model="gpt-4",
        #model="gpt-3.5-turbo",
        model="gpt-4-turbo",
        #model="gpt-4o-2024-05-13",
        #model="gpt-4-1106-preview",
        messages=[
            {
            "role": "system",
               "content": f"{prompt}. This is the text to process: {text}"
            },
        ],
        temperature=1,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return (response.choices[0].message.content)

In [89]:
def evaluate_ner_performance(df, ground_truth_col, model_col):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Default to empty list if None
        ground_truth = row[ground_truth_col] if row[ground_truth_col] else []
        predictions = row[model_col] if row[model_col] else []

        # Filter for 'PERSON' entities and standardize to 'PERSON' type
        ground_truth = [(start, end, 'PERSON') for start, end, typ in ground_truth if typ in ['PERSON', 'PER']]
        predictions = [(start, end, 'PERSON') for start, end, typ in predictions if typ in ['PERSON', 'PER']]

        # Sort by the start and then end positions
        gt_set = sorted(ground_truth, key=lambda x: (x[0], x[1]))
        pred_set = sorted(predictions, key=lambda x: (x[0], x[1]))

        # Calculate matches allowing for slight boundary mismatches
        matched_gt = set()
        matched_pred = set()

        for gt in gt_set:
            for pred in pred_set:
                # Check if entities match and boundaries are within tolerance
                if gt[2] == pred[2] and abs(gt[0] - pred[0]) <= 2 and abs(gt[1] - pred[1]) <= 2:
                    matched_gt.add(gt)
                    matched_pred.add(pred)

        # Update counts
        true_positives += len(matched_gt)
        false_positives += len(pred_set) - len(matched_pred)
        false_negatives += len(gt_set) - len(matched_gt)

    # Calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    print(true_positives, false_positives, false_negatives)

    return precision, recall, f1


In [287]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'rus_ner_model')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

3396 132 1805
Precision: 0.96, Recall: 0.65, F1 Score: 0.78


In [286]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'mult_model')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

3667 216 1534
Precision: 0.94, Recall: 0.71, F1 Score: 0.81


In [284]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'spacy')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

4206 783 995
Precision: 0.84, Recall: 0.81, F1 Score: 0.83


In [285]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'roberta_large')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

3988 341 1213
Precision: 0.92, Recall: 0.77, F1 Score: 0.84


In [181]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'gpt3.5-31-05-2024')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

3714 205 1487
Precision: 0.95, Recall: 0.71, F1 Score: 0.81


In [274]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'gpt4')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

3573 41 1628
Precision: 0.99, Recall: 0.69, F1 Score: 0.81


In [288]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'gpt4o')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

4492 177 709
Precision: 0.96, Recall: 0.86, F1 Score: 0.91


In [272]:
precision, recall, f1 = evaluate_ner_performance(df, 'labels', 'gpt4o_json')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

4693 214 508
Precision: 0.96, Recall: 0.90, F1 Score: 0.93
